In [58]:
%pip install pandas # to work with out data (load, clean, analyze)
%pip install dotenv # to load out enviroment variabels

84230.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


84238.93s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [59]:
# pip install pandas
import os # we need this for getting env variables here
import pandas as pd
from  dotenv import load_dotenv 
load_dotenv()

# load the csv file
dataset_path = os.getenv("DATASET_PATH")
movies_df = pd.read_csv(dataset_path)

In [60]:
# show the top 5 rows
movies_df.head(1)
movies_df.dtypes

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
backdrop_path            object
budget                    int64
homepage                 object
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity              float64
poster_path              object
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
dtype: object

In [61]:
# now I need the id,title, overview, tagline, genres
selected_columns = ["id","title","overview","vote_average","tagline","genres"]
movies_df_filtered = movies_df[selected_columns]
movies_df_filtered.head()


,id,title,overview,vote_average,tagline,genres
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...",8.364,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure"
1,157336,Interstellar,The adventures of a group of explorers who mak...,8.417,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction"
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,8.512,Welcome to a world without rules.,"Drama, Action, Crime, Thriller"
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",7.573,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction"
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,7.710,Some assembly required.,"Science Fiction, Action, Adventure"


In [62]:
movies_df.iloc[0,1]

print(movies_df.loc[0, "overview"])


Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.


### Before converting our data in to JSON format we need to clean the data.
- Preview data	df.head(), df.info()
- Remove unnecessary cols	
- Handle missing data	
- Fix types	
- Remove duplicates	
- Normalize values	
- Clean text	
- Verify	
- Save

In [63]:
movies_df_filtered.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239509 entries, 0 to 1239508
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1239509 non-null  int64  
 1   title         1239496 non-null  object 
 2   overview      974585 non-null   object 
 3   vote_average  1239509 non-null  float64
 4   tagline       173964 non-null   object 
 5   genres        722288 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 56.7+ MB


# Here’s what the info means in terms of nulls:

id — no nulls (all 1,239,509 entries have an id)

title — some nulls, since 1,239,496 < 1,239,509 (so about 13 missing)

overview — lots of nulls, only 974,585 non-null out of 1,239,509 (so about 264,924 missing)

vote_average - only 173964 avilable

tagline — even more nulls, only 173,964 non-null (about 1,065,545 missing)

genres — about half missing, 722,288 non-null (about 517,221 missing)



In [64]:
# Drop rows missing overview (must-have)
movies_df_filtered = movies_df_filtered.dropna(subset=['overview'])

# Fill missing tagline with empty string (lower priority)
movies_df_filtered['tagline'] = movies_df_filtered['tagline'].fillna('')

# Fill missing genres with empty list or 'unknown'
movies_df_filtered['genres'] = movies_df_filtered['genres'].fillna('unknown')

# Optional: fill missing titles if any
movies_df_filtered['title'] = movies_df_filtered['title'].fillna('Unknown Title')
movies_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 974585 entries, 0 to 1239507
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            974585 non-null  int64  
 1   title         974585 non-null  object 
 2   overview      974585 non-null  object 
 3   vote_average  974585 non-null  float64
 4   tagline       974585 non-null  object 
 5   genres        974585 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 52.0+ MB


In [65]:
movies_df_filtered["genres"]


0                   Action, Science Fiction, Adventure
1                    Adventure, Drama, Science Fiction
2                       Drama, Action, Crime, Thriller
3          Action, Adventure, Fantasy, Science Fiction
4                   Science Fiction, Action, Adventure
                              ...                     
1239499                                        unknown
1239501                               Animation, Music
1239502                                        unknown
1239503                           Documentary, History
1239507                                        unknown
Name: genres, Length: 974585, dtype: object

In [66]:
movies_df_filtered["tagline"]

0                       Your mind is the scene of the crime.
1          Mankind was born on Earth. It was never meant ...
2                          Welcome to a world without rules.
3                                Enter the world of Pandora.
4                                    Some assembly required.
                                 ...                        
1239499                                                     
1239501                                                     
1239502                                                     
1239503                                                     
1239507                                                     
Name: tagline, Length: 974585, dtype: object

In [67]:
movies_df.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."


In [68]:
movies_df_filtered.head(2)

,id,title,overview,vote_average,tagline,genres
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...",8.364,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure"
1,157336,Interstellar,The adventures of a group of explorers who mak...,8.417,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction"


now our dataset is almost ready to be converted to JSON. Beofre that we need maek the *"genre"* column JSON compatible.

we want [Action, Science Fiction, Adventure] to look like ["Action", "Science", "Fiction", "Adventure"] a list of str.



In [86]:
# split(',') methods remove the comma and make a list out of genre
# The strip() method removes any leading, and trailing whitespaces 

def clean_genres(genres_str):
    if isinstance(genres_str, str):
        # For null or missing values, return empty list
        if genres_str.lower() == "unknown" or not genres_str.strip():
            return []
        # Split by comma, strip spaces, and convert to lowercase
        return [g.strip().lower() for g in genres_str.split(',')]
    return []

# Apply it to your dataframe
movies_df_filtered['genres'] = movies_df_filtered['genres'].apply(clean_genres) 
movies_df_filtered.tail()    

,id,title,overview,vote_average,tagline,genres
1239499,783579,A Sunburnt Summer,Teddy wakes up from a hangover. He doesn't fee...,0.0,,[]
1239501,783581,The Stroke,The Stroke is a hand-drawn animation film that...,0.0,,[]
1239502,783582,Landry: Hair Chronicles,Landry’s energy is contagious. Coming to the D...,0.0,,[]
1239503,783584,On the Great War's menu: Food in the heart of ...,"In August 1914, war was declared. Millions of ...",0.0,,[]
1239507,783589,Transsexual Love Affair,Sometimes you want a little something on the s...,0.0,,[]


In [91]:
movies_df_filtered["genres"].info()

<class 'pandas.core.series.Series'>
Index: 974585 entries, 0 to 1239507
Series name: genres
Non-Null Count   Dtype 
--------------   ----- 
974585 non-null  object
dtypes: object(1)
memory usage: 14.9+ MB


# Before converting to JSON lets understand the relationship between JSON and Pthon

- `JSON object` → Python dict (mapping from string keys to values)

- `JSON array` → Python list

- `JSON string` → Python str

- `JSON numbe`r → Python int or float

- `JSON boolean` → Python bool

- `JSON null` → Python None

In [87]:

# Convert DataFrame rows to a list of dicts
movies_json_list = movies_df_filtered.to_dict(orient='records')

In [88]:
movies_json_list[55555]

{'id': 42762,
 'title': 'Hillsborough',
 'overview': "Drama based on the real life events of April 1989, when ninety-six Liverpool supporters were crushed to death during an F.A. Cup Semi-Final match against Nottingham Forest at Sheffield Wednesday's Hillsborough Stadium. This movie follows three Liverpudlian families before the match, during the tragedy and at the ensuing court battles which tried to decide who was to blame and what went wrong.",
 'vote_average': 7.1,
 'tagline': '',
 'genres': []}

In [ ]:
''' import json
with open('movies.json', 'w', encoding='utf-8') as f:
    json.dump(movies_json_list, f, ensure_ascii=False, indent=2) '''


KeyboardInterrupt: 